In [1]:
import random
import os
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait

from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np

In [2]:
import pandas as pd

In [3]:
MAX_WORKERS = 24

# Preprocessing

In [4]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...
2019496,3390742,199430,16,3,18,5.0,83,8,0,4,produce,fresh vegetables
2019497,458285,128787,42,2,19,3.0,115,1,1,7,beverages,water seltzer sparkling water
2019498,458285,128787,42,2,19,3.0,32,2,1,4,produce,packaged produce
2019499,458285,128787,42,2,19,3.0,32,3,1,4,produce,packaged produce


In [5]:
df = df[['order_id', 'user_id', 'order_number', 'product_id', 'product_name']]
df

,order_id,user_id,order_number,product_id,product_name
0,2425083,49125,1,17,baking ingredients
1,2425083,49125,1,91,soy lactosefree
2,2425083,49125,1,36,butter
3,2425083,49125,1,83,fresh vegetables
4,2425083,49125,1,83,fresh vegetables
...,...,...,...,...,...
2019496,3390742,199430,16,83,fresh vegetables
2019497,458285,128787,42,115,water seltzer sparkling water
2019498,458285,128787,42,32,packaged produce
2019499,458285,128787,42,32,packaged produce


In [6]:
df['product_name'].unique()

array(['baking ingredients', 'soy lactosefree', 'butter',
       'fresh vegetables', 'yogurt', 'canned meals beans',
       'poultry counter', 'ice cream ice', 'fresh fruits', 'milk',
       'packaged cheese', 'bread', 'tea', 'bakery desserts',
       'frozen breakfast', 'cereal', 'eggs', 'buns rolls', 'cream',
       'water seltzer sparkling water', 'pickled goods olives',
       'packaged poultry', 'other creams cheeses',
       'honeys syrups nectars', 'coffee', 'refrigerated',
       'energy granola bars', 'soft drinks', 'latino foods',
       'plates bowls cups flatware', 'paper goods', 'oral hygiene',
       'diapers wipes', 'food storage', 'nuts seeds dried fruit', 'soap',
       'packaged vegetables fruits', 'hot dogs bacon sausage',
       'lunch meat', 'chips pretzels', 'meat counter',
       'fresh dips tapenades', 'prepared soups salads', 'condiments',
       'juice nectars', 'canned fruit applesauce',
       'preserved dips spreads', 'packaged produce',
       'canned jarr

In [7]:
# return tuple {orderId, list(items)} for single check
def get_order_items(order_id):
    return (order_id, df[df['order_id'] == order_id]['product_name'].unique().tolist())

In [8]:
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:1000]):
    check_list.append(get_order_items(check_id))

  0%|          | 0/1000 [00:00<?, ?it/s]

# Layout

In [9]:
layout_test = Layout('./../data/layout 18x25_6.json').get_empty_rack_layout()
layout_test

In [10]:
def random_layout():
    layout = Layout('./../data/layout 18x25_6.json')
    layout.set_item_list(df['product_name'].unique().tolist())
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [11]:
layouts = []
for i in range(10):
    layouts.append(random_layout())

In [12]:
# dir = Path(os.getcwd()).parent
# layouts[2].display_in_window(home_dir=str(dir))

In [13]:
layouts[3]

# Evaluate

In [14]:
from genetic_algorithm_addons import evaluate_layout, thread_func

In [15]:
for layout in layouts:
    print(evaluate_layout(layout, check_list[0][1], use_item_count=True))

(60, False)
(72, False)
(74, False)
(70, False)
(78, False)
(66, False)
(70, False)
(0, True)
(64, False)
(82, False)


In [16]:
#dirr = Path(os.getcwd()).parent
#layouts[2].display_in_window(home_dir=str(dirr))

# Mutate layout

In [17]:
def select_n_best_layouts(layouts, checks, n_best=2, start_pos=None, reward_type='max', debug=False, use_item_count=False):
    res = dict()
    f_res = dict()
    
    if MAX_WORKERS > 1:
        futures = list()
        with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
            for layout in layouts:
                futures.append(executor.submit(thread_func, layout, checks, start_pos, use_item_count))
        
        #done, not_done = wait(futures, return_when='ALL_COMPLETED')
        for future in futures:
            # store modified layout as its path count is changed
            score, n_layout = future.result()
            res[n_layout] = score
    else:
        for layout in layouts:
            score, n_layout = thread_func(layout, checks, start_pos, use_item_count)
            res[n_layout] = score
    
    if reward_type == 'max':
        res = sorted(res.items(), key=lambda x: x[1][0], reverse=True)
    else:
        res = sorted(res.items(), key=lambda x: x[1][1])
    if debug:
        print([x[1] for x in res[:n_best]])
    return [x[0] for x in res[:n_best]]

In [18]:
new_layouts = select_n_best_layouts(layouts, check_list[:1], debug=True)

[(98, 0), (90, 0)]


In [19]:
new_layouts

In [20]:
def mutate_layout(original_layout, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        new_layout.set_item_to_rack(random.choice(new_layout.get_item_list()), (row, col), level=lev)
    return new_layout

In [21]:
temp = mutate_layout(new_layouts[0], alpha=0.1)

In [22]:
#dir = Path(os.getcwd()).parent
#temp.display_in_window(home_dir=str(dir))

# Genetic algorithm

In [32]:
layouts_array = []
for i in range(50):
    layouts_array.append(random_layout())

In [33]:
# evaluate, select, mutate, repeat
def genetic_algorithm(layouts_arr, check_arr, n_iter=100, n_best=10, alpha=0.01, start_pos=None, debug=False, reward_type='max', use_item_count=False):
    l_size = len(layouts_arr)
    for i in trange(n_iter):
        if debug:
            print(f'Iteration {i}', end=' ')
        n_best_arr = select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count)
        layouts_arr = []
        for l in n_best_arr:
            layouts_arr.append(l.copy().reset_path_count())
        for n in range(len(n_best_arr)):
            for m in range(l_size//n_best-1):
                layouts_arr.append(mutate_layout(n_best_arr[n], alpha=alpha))
    print(f'Iteration {n_iter}', end=' ')
    return select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count)

In [34]:
best_layouts = genetic_algorithm(layouts_array, check_list[:800], n_iter=75, n_best=10, alpha=0.03, debug=True, reward_type='max', use_item_count=True)

  0%|          | 0/75 [00:00<?, ?it/s]

Iteration 0 [(18780, 503), (15767, 530), (15625, 539), (14431, 560), (14414, 561), (14269, 551), (14234, 553), (14135, 560), (13970, 555), (13830, 555)]
Iteration 1 [(19341, 493), (18845, 503), (18580, 510), (18420, 511), (18342, 512), (15951, 536), (15937, 532), (15915, 542), (15823, 530), (15668, 539)]
Iteration 2 [(19839, 492), (19810, 493), (19641, 486), (19624, 490), (19391, 490), (19222, 493), (18864, 503), (18761, 503), (18753, 506), (18741, 490)]
Iteration 3 [(20906, 473), (20359, 485), (20338, 477), (20226, 480), (20166, 484), (20158, 478), (20123, 488), (20094, 480), (19966, 486), (19963, 479)]
Iteration 4 [(20851, 468), (20733, 473), (20662, 475), (20555, 481), (20544, 474), (20524, 478), (20511, 476), (20502, 474), (20423, 477), (20286, 483)]
Iteration 5 [(21602, 467), (21360, 464), (21336, 464), (21231, 463), (21171, 473), (21169, 467), (21121, 462), (20928, 473), (20847, 473), (20777, 475)]
Iteration 6 [(22203, 453), (22183, 460), (21857, 459), (21842, 460), (21680, 462),

In [35]:
curr_dir = Path(os.getcwd()).parent
best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=False)

In [24]:
for i in range(len(best_layouts)):
    best_layouts.save_to_json(f'./../data/layouts/best_layout_{i}.json')